## 埼玉県三郷市周辺の自転車盗難データを使ってヒートマップを作る 

In [16]:
## for spatial analysis
import geopandas as gpd

## for data analysis
import pandas as pd

## for pretty charts
import plotly.express as px

# for plotly themes
import plotly.io as pio

## for URL requests
import urllib.request
import requests

## for maps
import folium
from folium import plugins

In [17]:
df=pd.read_csv('Misatobike2022new.csv') 

In [18]:
misato=df

In [19]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   罪名             126 non-null    object
 1   手口             126 non-null    object
 2   管轄警察署（発生地）     126 non-null    object
 3   管轄交番・駐在所（発生地）  126 non-null    object
 4   市区町村コード（発生地）   126 non-null    int64 
 5   都道府県（発生地）      126 non-null    object
 6   市区町村（発生地）      126 non-null    object
 7   町丁目（発生地）       126 non-null    object
 8   発生年月日（始期）      126 non-null    object
 9   発生時（始期）        126 non-null    object
 10  発生場所           126 non-null    object
 11  発生場所の詳細        126 non-null    object
 12  被害者の年齢         126 non-null    object
 13  被害者の職業         126 non-null    object
 14  施錠関係           126 non-null    object
dtypes: int64(1), object(14)
memory usage: 14.9+ KB


In [20]:
#データをチェック
misato.sample(6)

,罪名,手口,管轄警察署（発生地）,管轄交番・駐在所（発生地）,市区町村コード（発生地）,都道府県（発生地）,市区町村（発生地）,町丁目（発生地）,発生年月日（始期）,発生時（始期）,発生場所,発生場所の詳細,被害者の年齢,被害者の職業,施錠関係
56,窃盗,自転車盗,吉川,三郷中央駅前交番,112372,埼玉県,三郷市,ピアラシティ１丁目,2022-09-04,17,その他,駐車（輪）場,10歳代,中学生,施錠せず
78,窃盗,自転車盗,吉川,三郷駅前交番,112372,埼玉県,三郷市,早稲田２丁目,2022-04-06,21,駐車（輪）場,駐車（輪）場,60-64歳,その他,施錠した
60,窃盗,自転車盗,吉川,三郷中央駅前交番,112372,埼玉県,三郷市,ピアラシティ１丁目,2022-11-14,10,その他,駐車（輪）場,10歳代,中学生,施錠した
92,窃盗,自転車盗,吉川,三郷駅前交番,112372,埼玉県,三郷市,早稲田４丁目,2022-07-15,10,４階建て以上共同住宅,駐車（輪）場,60-64歳,その他,施錠せず
11,窃盗,自転車盗,吉川,三郷中央駅前交番,112372,埼玉県,三郷市,花和田,2022-11-17,16,その他,駐車（輪）場,65-69歳,その他,施錠せず
28,窃盗,自転車盗,吉川,三郷中央駅前交番,112372,埼玉県,三郷市,中央１丁目,2022-04-30,7,駐車（輪）場,駐車（輪）場,20歳代,その他,施錠せず


# value_countsでチャートを作ってみる　1日に何時くらいに盗難されるのか。

In [21]:
# create a new variable with hourly counts
time = misato['発生時（始期）'].value_counts().reset_index()

time

,発生時（始期）,count
0,18,16
1,8,15
2,13,10
3,20,10
4,10,9
5,17,9
6,16,8
7,9,7
8,19,6
9,14,5


In [22]:
# fix headers
time.columns = ['発生時（始期）','件数']
time      

,発生時（始期）,件数
0,18,16
1,8,15
2,13,10
3,20,10
4,10,9
5,17,9
6,16,8
7,9,7
8,19,6
9,14,5


#### データフレームをチャートで表す

In [23]:
fig = px.bar(time,x='発生時（始期）',y='件数')
fig.show()

In [24]:
fig = px.bar(time,
            x='発生時（始期）',
            y='件数'
            )
fig.update_xaxes(categoryorder='category ascending')
fig.show()

In [25]:
fig = px.bar(time,
            x='発生時（始期）',
            y='件数',
            template= 'plotly_dark'
            )
fig.update_xaxes(categoryorder='category ascending')
fig.show()

In [26]:
fig = px.bar(misato,
            x='発生時（始期）',
            color='被害者の職業',
            template='seaborn')
fig.update_traces(
    marker_line_width=0
)
fig.update_xaxes(categoryorder='category ascending')
fig.show()

In [27]:
fig = px.bar(misato,
            x='発生時（始期）',
            color='被害者の職業',
            barmode='group', # group the categories,
            template='seaborn'
            )
fig.update_traces(
    marker_line_width=0
)
fig.update_xaxes(categoryorder='category ascending')
fig.show()

In [28]:
fig = px.bar(misato,
            x='発生時（始期）',
            facet_col='被害者の職業',
            template='seaborn')
fig.update_traces(
    marker_line_width=0
)
fig.update_xaxes(categoryorder='category ascending')
fig.show()

## これらのデータをジオコーディングする　

In [29]:
# 関数を作成
def geocode(address):

    # ジオコーティングURL
    url = "https://msearch.gsi.go.jp/address-search/AddressSearch?q="

    # try/exceptでエラーをキャッチ
    # 成功の場合
    try:
        s_quote = urllib.parse.quote(address)
        response = requests.get(url + s_quote)
        if len(response.json())>0:
            return response.json()[0]["geometry"]["coordinates"] 
        else:
            return False
        
    # 失敗の場合
    except:
        return False

In [30]:
geocode('三郷市')

[139.872284, 35.830151]

In [31]:
# 空の住所フィールドを作成
misato['住所'] = ''

In [32]:
# 空だがfloatとしてフィールドを作成
misato['lat'] = pd.Series(dtype=float)
misato['lon'] = pd.Series(dtype=float)

In [33]:
# 住所フィールドを作成
misato['住所'] = misato['都道府県（発生地）']+misato['市区町村（発生地）']+misato['町丁目（発生地）']

In [34]:
# random sampleで上手く行ったかどうかをチェック
misato.sample(3)

,罪名,手口,管轄警察署（発生地）,管轄交番・駐在所（発生地）,市区町村コード（発生地）,都道府県（発生地）,市区町村（発生地）,町丁目（発生地）,発生年月日（始期）,発生時（始期）,発生場所,発生場所の詳細,被害者の年齢,被害者の職業,施錠関係,住所,lat,lon
86,窃盗,自転車盗,吉川,三郷駅前交番,112372,埼玉県,三郷市,早稲田２丁目,2022-12-26,21,駐車（輪）場,駐車（輪）場,20歳代,その他,施錠せず,埼玉県三郷市早稲田２丁目,NaN,NaN
23,窃盗,自転車盗,吉川,三郷中央駅前交番,112372,埼玉県,三郷市,谷口,2022-06-04,22,その他の住宅（３階建て以下共同住宅等）,駐車（輪）場,20歳代,その他,施錠せず,埼玉県三郷市谷口,NaN,NaN
92,窃盗,自転車盗,吉川,三郷駅前交番,112372,埼玉県,三郷市,早稲田４丁目,2022-07-15,10,４階建て以上共同住宅,駐車（輪）場,60-64歳,その他,施錠せず,埼玉県三郷市早稲田４丁目,NaN,NaN


In [35]:
# kashiwaデータフレームをループ
for i,row in misato.iterrows():

    # ジオコーティング成功
    if geocode(row['住所']) != False:
        # 座標を変数に
        lon = geocode(row['住所'])[0]
        lat = geocode(row['住所'])[1]

        # データフレームに値をインプット        
        misato.loc[i,'lon'] = lon
        misato.loc[i,'lat'] = lat

        # 結果をprint
        print(row['住所'],lon,lat)
    
    # ジオコーティング失敗
    else:
        print(row['住所'],'ジオコーティング失敗')
        continue

埼玉県三郷市泉２丁目 139.860626 35.843063
埼玉県三郷市泉２丁目 139.860626 35.843063
埼玉県三郷市岩野木 139.886154 35.833054
埼玉県三郷市幸房 139.879288 35.837543
埼玉県三郷市新和４丁目 139.885666 35.811974
埼玉県三郷市新和４丁目 139.885666 35.811974
埼玉県三郷市新和４丁目 139.885666 35.811974
埼玉県三郷市新和４丁目 139.885666 35.811974
埼玉県三郷市花和田 139.86026 35.826832
埼玉県三郷市花和田 139.86026 35.826832
埼玉県三郷市花和田 139.86026 35.826832
埼玉県三郷市花和田 139.86026 35.826832
埼玉県三郷市花和田 139.86026 35.826832
埼玉県三郷市花和田 139.86026 35.826832
埼玉県三郷市彦江１丁目 139.85405 35.82724
埼玉県三郷市彦江１丁目 139.85405 35.82724
埼玉県三郷市彦江１丁目 139.85405 35.82724
埼玉県三郷市彦江１丁目 139.85405 35.82724
埼玉県三郷市彦江１丁目 139.85405 35.82724
埼玉県三郷市彦野１丁目 139.848221 35.841343
埼玉県三郷市彦野１丁目 139.848221 35.841343
埼玉県三郷市谷口 139.870361 35.826015
埼玉県三郷市谷口 139.870361 35.826015
埼玉県三郷市谷口 139.870361 35.826015
埼玉県三郷市中央１丁目 139.876038 35.822933
埼玉県三郷市中央１丁目 139.876038 35.822933
埼玉県三郷市中央１丁目 139.876038 35.822933
埼玉県三郷市中央１丁目 139.876038 35.822933
埼玉県三郷市中央１丁目 139.876038 35.822933
埼玉県三郷市中央１丁目 139.876038 35.822933
埼玉県三郷市中央１丁目 139.876038 35.822933
埼玉県三郷市中央１丁目 139.87603

In [36]:
misato.sample(5)

,罪名,手口,管轄警察署（発生地）,管轄交番・駐在所（発生地）,市区町村コード（発生地）,都道府県（発生地）,市区町村（発生地）,町丁目（発生地）,発生年月日（始期）,発生時（始期）,発生場所,発生場所の詳細,被害者の年齢,被害者の職業,施錠関係,住所,lat,lon
108,窃盗,自転車盗,吉川,三郷駅前交番,112372,埼玉県,三郷市,三郷１丁目,2022-01-31,14,駐車（輪）場,駐車（輪）場,40歳代,その他,施錠した,埼玉県三郷市三郷１丁目,35.841259,139.886047
8,窃盗,自転車盗,吉川,三郷中央駅前交番,112372,埼玉県,三郷市,花和田,2022-05-19,8,その他,駐車（輪）場,30歳代,その他,施錠せず,埼玉県三郷市花和田,35.826832,139.860260
84,窃盗,自転車盗,吉川,三郷駅前交番,112372,埼玉県,三郷市,早稲田２丁目,2022-09-04,17,一戸建住宅,駐車（輪）場,50歳代,その他,施錠せず,埼玉県三郷市早稲田２丁目,35.848438,139.882874
55,窃盗,自転車盗,吉川,三郷中央駅前交番,112372,埼玉県,三郷市,ピアラシティ１丁目,2022-08-28,18,その他,駐車（輪）場,20歳代,その他,施錠せず,埼玉県三郷市ピアラシティ１丁目,35.842663,139.855316
74,窃盗,自転車盗,吉川,三郷駅前交番,112372,埼玉県,三郷市,早稲田１丁目,2022-06-11,5,駐車（輪）場,駐車（輪）場,30歳代,その他,施錠せず,埼玉県三郷市早稲田１丁目,35.847256,139.887070


In [37]:
# 全データの中央座標
center_lat = misato.lat.mean()
center_lon = misato.lon.mean() 

In [38]:
# make the map
m = folium.Map(location=[center_lat,center_lon], 
               zoom_start=12)

# マーカーがメチャクチャ多い場合はclusterで処理！
marker_cluster = plugins.MarkerCluster().add_to(m)
 
# kashiwaのデータフレームをループしてマーカーを作る
for index, row in misato.iterrows():
    latlon = [row['lat'],row['lon']]
    folium.Marker(latlon, 
                  tooltip=row['被害者の年齢'],
                ).add_to(marker_cluster) # mapにではなくmarker_clusterに足す

# show the map
m

## Heat Map in Misato city area

In [39]:
# make a list of lat/lon's
heatmap_lat_lon = misato[['lat','lon']].values.tolist()

# view the list
heatmap_lat_lon

[[35.843063, 139.860626],
 [35.843063, 139.860626],
 [35.833054, 139.886154],
 [35.837543, 139.879288],
 [35.811974, 139.885666],
 [35.811974, 139.885666],
 [35.811974, 139.885666],
 [35.811974, 139.885666],
 [35.826832, 139.86026],
 [35.826832, 139.86026],
 [35.826832, 139.86026],
 [35.826832, 139.86026],
 [35.826832, 139.86026],
 [35.826832, 139.86026],
 [35.82724, 139.85405],
 [35.82724, 139.85405],
 [35.82724, 139.85405],
 [35.82724, 139.85405],
 [35.82724, 139.85405],
 [35.841343, 139.848221],
 [35.841343, 139.848221],
 [35.826015, 139.870361],
 [35.826015, 139.870361],
 [35.826015, 139.870361],
 [35.822933, 139.876038],
 [35.822933, 139.876038],
 [35.822933, 139.876038],
 [35.822933, 139.876038],
 [35.822933, 139.876038],
 [35.822933, 139.876038],
 [35.822933, 139.876038],
 [35.822933, 139.876038],
 [35.822933, 139.876038],
 [35.822933, 139.876038],
 [35.822933, 139.876038],
 [35.822933, 139.876038],
 [35.822933, 139.876038],
 [35.822933, 139.876038],
 [35.822933, 139.876038],
 [

In [40]:
from folium.plugins import HeatMap

# make the map
m = folium.Map(location=[center_lat,center_lon], 
               zoom_start=12,
               tiles='cartodbdark_matter')

# add the heatmap
HeatMap(
    data = heatmap_lat_lon,
    radius=15,    
).add_to(m)

# マーカーがメチャクチャ多い場合はclusterで処理！
marker_cluster = plugins.MarkerCluster().add_to(m)

# kashiwaのデータフレームをループしてマーカーを作る
for index, row in misato.iterrows():
    latlon = [row['lat'],row['lon']]
    folium.Marker(latlon, 
                  tooltip=row['被害者の年齢'],
                ).add_to(marker_cluster) # mapにではなくmarker_clusterに足す

# show the map
m

### 気づいたこと　

今回は自分が住んでいる三郷市がどのように自転車の盗難があるかデータを集め、ジオコーディングをした。
駅には自転車を停める場所や店の前で自転車を停める人がいるので、自転車の盗難が最も多く起きているエリアだということがマップを通して分かった。
そして、データでは学生は学校が終わった後の帰宅時間に盗難被害に遭いやすく、学生以外の社会人は夜や朝方が多いということが分かった。
データだけでも、年齢や時間があれば、どういう人がどういう行動を起こしているのかが想像しやすいと感じた。　